#### Name: Tony .U. Akalonu 


### Python program to generate fibonacci sequence


In [1]:
n = int (input("Enter the number of terms needed in the Fibonacci series: "))

if (n<0):
    print ("Enter a positive number")
else:
    f1, f2 = 0, 1
    if n == 1:
        print (f1)
    elif n == 2:
        print (f1,f2)
    else:
        print (f1,f2, end = ' ')
        for i in range (3, n+1):
            f3 = f1 + f2
            print (f3, end = ' ')
            f1 = f2
            f2 = f3

Enter the number of terms needed in the Fibonacci series: 20
0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 

### TO-DO LIST APP AND CONNECTING IT TO POSTGRESQL


In [2]:
import tkinter
import tkinter.messagebox
import pickle
import psycopg2
from psycopg2 import Error

root = tkinter.Tk()
root.title("To-Do List by Tony Akalonu")

def add_task():
    task = entry_task.get()
    if task != "":
        listbox_tasks.insert(tkinter.END, task)
        entry_task.delete(0, tkinter.END)
    else:
        tkinter.messagebox.showwarning(title="Warning!", message="You must enter a task.")


def delete_task():
    try:
        task_index = listbox_tasks.curselection()[0]
        listbox_tasks.delete(task_index)
    except:
        tkinter.messagebox.showwarning(title="Warning!", message="You must select a task.")


def cross_out_task():
    try:
        task_index = listbox_tasks.curselection()[0]
        task_text = listbox_tasks.get(task_index)
        # Check if the task is already crossed out
        if not task_text.startswith("✓ "):
            # Cross out the task
            listbox_tasks.delete(task_index)
            listbox_tasks.insert(task_index, "✓ " + task_text)
    except:
        tkinter.messagebox.showwarning(title="Warning!", message="You must select a task.")


def uncross_task():
    try:
        task_index = listbox_tasks.curselection()[0]
        task_text = listbox_tasks.get(task_index)
        # Check if the task is crossed out
        if task_text.startswith("✓ "):
            # Uncross the task
            listbox_tasks.delete(task_index)
            listbox_tasks.insert(task_index, task_text[2:])
    except:
        tkinter.messagebox.showwarning(title="Warning!", message="You must select a task.")


def load_tasks():
    try:
        connection = psycopg2.connect(
            user="bincom_todo_list",
            password="****",
            host="127.0.0.1",
            port="5432",
            database="bincom_todo_list_db"
        )
        cursor = connection.cursor()
        cursor.execute("SELECT task FROM tasks")
        tasks = cursor.fetchall()
        listbox_tasks.delete(0, tkinter.END)
        for task in tasks:
            listbox_tasks.insert(tkinter.END, task[0])
        cursor.close()
        connection.close()
    except (Exception, Error) as e:
        tkinter.messagebox.showwarning(title="Warning!", message="An error occurred while loading tasks from the database.")
        print("Error:", str(e))


def save_tasks():
    tasks = listbox_tasks.get(0, tkinter.END)
    try:
        connection = psycopg2.connect(
            user="bincom_todo_list",
            password="****",
            host="127.0.0.1",
            port="5432",
            database="bincom_todo_list_db"
        )
        cursor = connection.cursor()
        cursor.execute("DELETE FROM tasks")
        for task in tasks:
            cursor.execute("INSERT INTO tasks (task) VALUES (%s)", (task,))
        connection.commit()
        cursor.close()
        connection.close()
        tkinter.messagebox.showinfo(title="Success", message="Tasks saved successfully to the database.")
    except (Exception, Error) as e:
        tkinter.messagebox.showerror(title="Error", message="An error occurred while saving tasks to the database.")
        print("Error:", str(e))


# Create GUI
frame_tasks = tkinter.Frame(root)
frame_tasks.pack()
# list box to show all the tasks
listbox_tasks = tkinter.Listbox(frame_tasks, height=10, width=50)
listbox_tasks.pack(side=tkinter.LEFT)
#scroll bar on the right side
scrollbar_tasks = tkinter.Scrollbar(frame_tasks)
scrollbar_tasks.pack(side=tkinter.RIGHT, fill=tkinter.Y)
# connecting the list box and the scroll bar
listbox_tasks.config(yscrollcommand=scrollbar_tasks.set)# listbox task and widget are connected using the yscrollcommand
scrollbar_tasks.config(command=listbox_tasks.yview)
# where we type our entry into
entry_task = tkinter.Entry(root, width=50)
entry_task.pack()
#add task button
button_add_task = tkinter.Button(root, text="Add task", width=48, command=add_task)
button_add_task.pack()
#button to delete task
button_delete_task = tkinter.Button(root, text="Delete task", width=48, command=delete_task)
button_delete_task.pack()
#button to cross out task as done
button_cross_out_task = tkinter.Button(root, text="Cross out task", width=48, command=cross_out_task)
button_cross_out_task.pack()
#button to uncross out task
button_uncross_task = tkinter.Button(root, text="Uncross task", width=48, command=uncross_task)
button_uncross_task.pack()
#button to load task
button_load_tasks = tkinter.Button(root, text="Load tasks", width=48, command=load_tasks)
button_load_tasks.pack()
#button to save task
button_save_tasks = tkinter.Button(root, text="Save tasks", width=48, command=save_tasks)
button_save_tasks.pack()

root.mainloop()
